In [8]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json

In [9]:
# Local on Windows 10 box
WD = r"D:\BUSTEDS-MH"
BUSTEDS_DIR = os.path.join(WD, r"analysis\13-datasets\BUSTEDS")
BUSTEDS_MH_DIR = os.path.join(WD, r"analysis\13-datasets\BUSTEDS-MH")
BUSTEDS_OUTPUT_CSV = os.path.join(WD, r"Processed_13Datasets_BUSTEDS.csv")
BUSTEDS_MH_OUTPUT_CSV = os.path.join(WD, r"Processed_13Datasets_BUSTEDS-MH.csv")
OUTPUT_CSV = os.path.join(WD, r"Table_Processed_13Datasets_BUSTEDS_vs_BUSTEDS-MH.csv")

In [91]:
def read_json(filename):
    print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method


#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)

In [92]:
BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".json")]
BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]

print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))


# Number of BUSTEDS results: 13
# Number of BUSTEDS-MH results: 13


In [93]:
BUSTEDS_MH_DIR_FILES

['D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\adh.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\bglobin.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\camelid.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\COXI.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\ENCenv.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\flavNS5.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\HepatitisD.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\HIVvif.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\HIV_RT.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\InfluenzaA.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\lysin.nex.BUSTEDS-MH.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS-MH\\lysozyme.nex.BUSTEDS-MH.json',
 'D:\\BUS

In [94]:
BUSTEDS_DIR_FILES

['D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\adh.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\bglobin.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\camelid.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\COXI.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\ENCenv.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\flavNS5.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\HepatitisD.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\HIVvif.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\HIV_RT.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\InfluenzaA.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\lysin.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\lysozyme.nex.BUSTEDS.json',
 'D:\\BUSTEDS-MH\\analysis\\13-datasets\\BUSTEDS\\yokoyama.rh1.cds.mod.1-990.nex.

In [136]:
df_dict = {}

for item in BUSTEDS_MH_DIR_FILES:
    basename = os.path.basename(item).replace(".nex.BUSTEDS-MH.json", "")
    
    # Find BUSTEDS File
    BUSTEDS_File = os.path.join(BUSTEDS_DIR, basename + ".nex.BUSTEDS.json")
    
    # Read json
    json_data_BUSTEDS_MH = read_json(item)
    json_data_BUSTEDS = read_json(BUSTEDS_File)
    
    print("# Data loaded")
    df_dict[basename] = {"Sequences": json_data_BUSTEDS_MH["input"]["number of sequences"]}
    df_dict[basename].update({"Codons": json_data_BUSTEDS_MH["input"]["number of sites"]})
    df_dict[basename].update({"PValue_BUSTEDS-MH": json_data_BUSTEDS_MH["test results"]["p-value"]})

    A = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    
    df_dict[basename].update({"CV(omega)_BUSTEDS-MH": cv([A, B, C])})
    
    D = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
  
    df_dict[basename].update({"CV(alpha)_BUSTEDS-MH": cv([D, E, F])})
    
    df_dict[basename].update({"omega_3_BUSTEDS-MH": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["omega"]})
    df_dict[basename].update({"proportion_3_BUSTEDS-MH": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["proportion"]})
    
    # DH rate, TH rate
    df_dict[basename].update({"DH_Rate": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 2 nucleotides are changed instantly within a single codon"]})
    df_dict[basename].update({"TH_Rate": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon"]})
    df_dict[basename].update({"TH_Rate_SI": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"]})
    
    # BUSTEDS
    df_dict[basename].update({"PValue_BUSTEDS": json_data_BUSTEDS["test results"]["p-value"]})
    
    A = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    
    df_dict[basename].update({"CV(omega)_BUSTEDS": cv([A, B, C])})
    
    D = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    
    df_dict[basename].update({"CV(alpha)_BUSTEDS": cv([D, E, F])})
    df_dict[basename].update({"omega_3_BUSTEDS": json_data_BUSTEDS["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["omega"]})
    df_dict[basename].update({"proportion_3_BUSTEDS": json_data_BUSTEDS["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["proportion"]})
    
    # cAIC
    AICc_BUSTEDS_MH = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["AIC-c"]
    AICc_BUSTEDS = json_data_BUSTEDS["fits"]["Unconstrained model"]["AIC-c"]
    print("# Calculating 'Unconstrained model' AICc delta, (BUSTEDS-MH)", AICc_BUSTEDS_MH, "and (BUSTEDS)", AICc_BUSTEDS)
    delta_AICc = AICc_BUSTEDS_MH - AICc_BUSTEDS
    df_dict[basename].update({"delta_cAIC": delta_AICc})
    
    # ER Sites > 5
    ER_SITES_BUSTEDS = []
    ER_SITES_BUSTEDS_MH = []
    
    if "constrained" in json_data_BUSTEDS_MH["Evidence Ratios"].keys():
        print("# ER Constrained:", len(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]):
            if val > 5:
                ER_SITES_BUSTEDS_MH.append(site + 1)
            #end if
        #end for
    #end if
    
    if "constrained" in json_data_BUSTEDS["Evidence Ratios"].keys():
        print("# ER Constrained:", len(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]):
            if val > 5:
                ER_SITES_BUSTEDS.append(site + 1)
            #end if
        #end for
    #end if
    
    print("# BUSTEDS-MH:",ER_SITES_BUSTEDS_MH, len(ER_SITES_BUSTEDS_MH))
    print("# BUSTEDS:", ER_SITES_BUSTEDS, len(ER_SITES_BUSTEDS))
    
    union_of_sites = matches = [item for item in ER_SITES_BUSTEDS_MH if item in ER_SITES_BUSTEDS]
    print(union_of_sites, len(union_of_sites))
    
    # In BUSTEDS-MH and BUSTEDS
    df_dict[basename].update({"++": len(union_of_sites)})
    
    # In BUSTEDS-MH and NOT BUSTEDS
    df_dict[basename].update({"+-": abs(len(ER_SITES_BUSTEDS_MH) - len(union_of_sites))})
    
    # NOT in BUSTEDS-MH but in BUSTEDS
    df_dict[basename].update({"-+": abs(len(ER_SITES_BUSTEDS) - len(union_of_sites))})
    

# Reading: D:\BUSTEDS-MH\analysis\13-datasets\BUSTEDS-MH\adh.nex.BUSTEDS-MH.json
# Reading: D:\BUSTEDS-MH\analysis\13-datasets\BUSTEDS\adh.nex.BUSTEDS.json
# Data loaded
# Calculating 'Unconstrained model' AICc delta, (BUSTEDS-MH) 9359.172806510542 and (BUSTEDS) 9353.863170589633
# ER Constrained: 254
# ER Constrained: 254
# BUSTEDS-MH: [6, 35, 39, 49, 69, 133, 134, 163, 165, 166, 170, 197, 216, 227, 253] 15
# BUSTEDS: [4, 6, 35, 39, 46, 49, 69, 133, 134, 163, 165, 166, 170, 187, 197, 216, 227, 253] 18
[6, 35, 39, 49, 69, 133, 134, 163, 165, 166, 170, 197, 216, 227, 253] 15
# Reading: D:\BUSTEDS-MH\analysis\13-datasets\BUSTEDS-MH\bglobin.nex.BUSTEDS-MH.json
# Reading: D:\BUSTEDS-MH\analysis\13-datasets\BUSTEDS\bglobin.nex.BUSTEDS.json
# Data loaded
# Calculating 'Unconstrained model' AICc delta, (BUSTEDS-MH) 7384.667181616638 and (BUSTEDS) 7415.590435899347
# ER Constrained: 144
# ER Constrained: 144
# BUSTEDS-MH: [3, 8, 10, 11, 21, 42, 48, 50, 54, 74, 110, 114, 116, 124, 133] 15
# BUS

## Convert to dataframe

In [146]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df = df.reset_index()
df.index += 1
df.rename(columns={'index': 'Gene'}, inplace = True)
df

,Gene,Sequences,Codons,PValue_BUSTEDS-MH,CV(omega)_BUSTEDS-MH,CV(alpha)_BUSTEDS-MH,omega_3_BUSTEDS-MH,proportion_3_BUSTEDS-MH,DH_Rate,TH_Rate,TH_Rate_SI,PValue_BUSTEDS,CV(omega)_BUSTEDS,CV(alpha)_BUSTEDS,omega_3_BUSTEDS,proportion_3_BUSTEDS,delta_cAIC,++,+-,-+
1,adh,23,254,0.019474,1.371220,0.535276,3.932678,0.024832,0.000000,0.033881,3.375762,1.639209e-03,1.363391,0.535276,4.355042,0.023882,5.309636,15,0,3
2,bglobin,17,144,0.002195,1.357236,1.133289,9.833868,0.030157,0.000000,0.000000,260.314688,1.182876e-05,1.350053,1.133289,8.763965,0.037515,-30.923254,15,0,5
3,camelid,212,96,0.005554,1.036795,0.822342,6.125249,0.044567,0.162711,0.000000,0.000000,0.000000e+00,1.331429,0.822342,24.630822,0.020229,-26.241272,21,1,12
4,COXI,21,510,0.500000,1.402459,1.316585,1.000000,0.011291,0.000000,0.000000,4.350649,5.000000e-01,1.389798,1.316585,1.084193,0.009435,6.813631,0,0,0
5,ENCenv,23,500,0.500000,1.204253,0.895045,1.001563,0.000000,0.012858,0.000000,0.000000,5.000000e-01,1.154258,0.895045,1.176761,0.000000,6.031367,0,0,0
6,flavNS5,18,342,0.500000,1.347522,1.216725,1.075472,0.000000,0.364627,0.902814,2.386362,4.822039e-01,1.362664,1.216725,1.165364,0.019089,-41.953286,0,0,0
7,HepatitisD,33,196,0.032277,1.302968,0.974883,12.047043,0.015463,0.144653,0.000000,0.000000,1.118888e-08,1.332831,0.974883,16.587812,0.019107,-5.961838,7,0,16
8,HIVvif,29,192,0.500000,0.474882,0.827973,1.804586,0.000000,0.003737,0.163413,0.000000,2.270883e-02,1.413321,0.827973,1802.234727,0.000480,-0.640757,0,0,1
9,HIV_RT,476,335,0.001198,1.407424,0.836829,47.059883,0.000688,0.039494,0.000000,0.000000,7.174705e-11,1.407637,0.836829,48.964132,0.001009,2.330108,12,0,10
10,InfluenzaA,349,329,0.500000,0.433551,0.870295,1.002331,0.000000,0.062816,0.014894,0.000000,1.047128e-01,1.414214,0.870295,1.580507,0.289351,-12.388686,0,0,31


In [152]:
# Save dataframe

# saving the dataframe
df.to_csv('Analysis_13Datasets.csv')


In [76]:
# Figure legend

"""

Results from the reanalysis of the data sets used in Kosakovsky Pond and Muse (2005), 
Yokoyama et al. (2008), and Chen and Sun (2011), 
arranged by sequence count. 

We ran selection tests with three nonsynonymous and three synonymous rate categories (for BUSTED[S]). 
Column headings are as follows: S, number of sequences; N, number of codons; CV(ω)⁠, 
the coefficient of variation (CV) for the inferred distribution of ω ratios; ω3, 
the maximum likelihood estimate (MLE) of the strength of selection; p3,
the MLE of the proportion of sites under selection (proportion of sites in the ω3 category); 
CV(α)⁠, the CV for the inferred distribution of synonymous rates; ΔAICc, the difference between AICc values of BUSTED and BUSTED[S]. 
The α distribution columns list the estimated values of the 3 α categories along with their estimated frequencies. 
The Sites columns count the number of alignments where at least one method called a site selected (using evidence ratio of at least 5): 
++ both methods yes; +− BUSTED yes, BUSTED[S] no; −+ BUSTED no, BUSTED[S] yes.

"""

'\n\nResults from the reanalysis of the data sets used in Kosakovsky Pond and Muse (2005), \nYokoyama et al. (2008), and Chen and Sun (2011), \narranged by sequence count. \n\nWe ran selection tests with three nonsynonymous and three synonymous rate categories (for BUSTED[S]). \nColumn headings are as follows: S, number of sequences; N, number of codons; CV(ω)\u2060, \nthe coefficient of variation (CV) for the inferred distribution of ω ratios; ω3, \nthe maximum likelihood estimate (MLE) of the strength of selection; p3,\nthe MLE of the proportion of sites under selection (proportion of sites in the ω3 category); \nCV(α)\u2060, the CV for the inferred distribution of synonymous rates; ΔAICc, the difference between AICc values of BUSTED and BUSTED[S]. \nThe α distribution columns list the estimated values of the 3 α categories along with their estimated frequencies. \nThe Sites columns count the number of alignments where at least one method called a site selected (using evidence ratio 

In [66]:
# Note Negative delta LL are convergence problems

In [ ]:
# Lower AIC values indicate a better-fit model, and a model with a delta-AIC (the difference between the two AIC values being compared) of more than -2 is considered significantly better than the model it is being compared to

In [120]:
# Earth Mover's (Kantorovich) distance between two distrbuitions if you want a single number

In [147]:
df = df.sort_values(by="Sequences", ascending=False)
df = df.reset_index(drop=True)
df.index += 1
styled_table = df.style.background_gradient()
styled_table

,Gene,Sequences,Codons,PValue_BUSTEDS-MH,CV(omega)_BUSTEDS-MH,CV(alpha)_BUSTEDS-MH,omega_3_BUSTEDS-MH,proportion_3_BUSTEDS-MH,DH_Rate,TH_Rate,TH_Rate_SI,PValue_BUSTEDS,CV(omega)_BUSTEDS,CV(alpha)_BUSTEDS,omega_3_BUSTEDS,proportion_3_BUSTEDS,delta_cAIC,++,+-,-+
1,HIV_RT,476,335,0.001198,1.407424,0.836829,47.059883,0.000688,0.039494,0.000000,0.000000,0.000000,1.407637,0.836829,48.964132,0.001009,2.330108,12,0,10
2,InfluenzaA,349,329,0.500000,0.433551,0.870295,1.002331,0.000000,0.062816,0.014894,0.000000,0.104713,1.414214,0.870295,1.580507,0.289351,-12.388686,0,0,31
3,camelid,212,96,0.005554,1.036795,0.822342,6.125249,0.044567,0.162711,0.000000,0.000000,0.000000,1.331429,0.822342,24.630822,0.020229,-26.241272,21,1,12
4,yokoyama.rh1.cds.mod.1-990,38,330,0.089758,1.362607,1.023490,5.518101,0.006365,0.243119,0.280597,11.359972,0.000001,1.371595,1.023490,7.037441,0.011822,-28.356389,2,0,27
5,HepatitisD,33,196,0.032277,1.302968,0.974883,12.047043,0.015463,0.144653,0.000000,0.000000,0.000000,1.332831,0.974883,16.587812,0.019107,-5.961838,7,0,16
6,HIVvif,29,192,0.500000,0.474882,0.827973,1.804586,0.000000,0.003737,0.163413,0.000000,0.022709,1.413321,0.827973,1802.234727,0.000480,-0.640757,0,0,1
7,lysin,25,134,0.000001,1.300578,0.873833,17.391589,0.075910,0.000000,0.000000,0.000000,0.000000,1.232795,0.873833,15.833466,0.091965,1.971931,34,0,9
8,adh,23,254,0.019474,1.371220,0.535276,3.932678,0.024832,0.000000,0.033881,3.375762,0.001639,1.363391,0.535276,4.355042,0.023882,5.309636,15,0,3
9,ENCenv,23,500,0.500000,1.204253,0.895045,1.001563,0.000000,0.012858,0.000000,0.000000,0.500000,1.154258,0.895045,1.176761,0.000000,6.031367,0,0,0
10,COXI,21,510,0.500000,1.402459,1.316585,1.000000,0.011291,0.000000,0.000000,4.350649,0.500000,1.389798,1.316585,1.084193,0.009435,6.813631,0,0,0


In [131]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas.table.plotting import table # EDIT: see deprecation warnings below

ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, df)  # where df is your data frame

plt.savefig('mytable.png')

ModuleNotFoundError: No module named 'pandas.table'

In [135]:
json_data_BUSTEDS_MH["fits"]["Unconstrained model"].keys()

dict_keys(['AIC-c', 'Log Likelihood', 'Rate Distributions', 'display order', 'estimated parameters', 'rate at which 2 nucleotides are changed instantly within a single codon', 'rate at which 3 nucleotides are changed instantly within a single codon', 'rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands'])

In [151]:
import plotly.figure_factory as ff
import pandas as pd

fig =  ff.create_table(df)
fig.update_layout(
    autosize=False,
    width=500,
    height=200,
)
fig.write_image("table_plotly.png", scale=2)
fig.show()

ValueError: Image generation requires the psutil package.

Install using pip:
    $ pip install psutil

Install using conda:
    $ conda install psutil
